In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

### DATASET AVEC STOPWORDS OCCURENCES/TF-IDF

In [2]:
train_text = pd.read_csv('../Ressources/separated_data/scenarii_training_text_sans_chiffres.csv', encoding="ISO-8859-1", header = None)
train_tags = pd.read_csv('../Ressources/separated_data/scenarii_training_tags_sans_chiffres.csv', encoding="ISO-8859-1", header = None)
test_text = pd.read_csv('../Ressources/separated_data/scenarii_testing_text_sans_chiffres.csv', encoding="ISO-8859-1", header = None)
test_tags = pd.read_csv('../Ressources/separated_data/scenarii_testing_tags_sans_chiffres.csv', encoding="ISO-8859-1", header = None)

train_text = train_text[train_text.columns[0]]
train_tags = train_tags[train_tags.columns[0]]
test_text = test_text[test_text.columns[0]]
test_tags = test_tags[test_tags.columns[0]]

In [3]:
text_clf = Pipeline([('vect', CountVectorizer()),
                         #Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
text_clf.fit(train_text, train_tags)
predicted = text_clf.predict(test_text)
print("Precision: " + str(precision_score(test_tags, predicted, average='macro')))
print("Recall: " + str(recall_score(test_tags, predicted, average='macro')))
print("F measure: " + str(f1_score(test_tags, predicted, average='macro')))

Precision: 0.7245898769676968
Recall: 0.47185381800906523
F measure: 0.4735791673931516


#### CROSS VALIDATION

In [4]:
df = pd.read_csv("../Ressources/scenarii.csv", encoding="ISO-8859-1", sep=";", header=0)
tags = df['tag']
data = CountVectorizer().fit_transform(df['texte']).toarray()
#Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
data = TfidfTransformer().fit_transform(data).toarray()

In [5]:
clf = MultinomialNB()
scoring = {'accuracy': 'accuracy',
           'prec': 'average_precision', 
           'recall': 'recall',
           'f1_macro': 'f1_macro'}
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'f1_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'recall_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'precision_macro')
print(np.mean(scores))

0.594572949697851
0.5492801993328309
0.8668031953683976


### DATASET SANS STOPWORDS OCCURENCES/TF-IDF


In [6]:
'''
Construction du modèle Naive Bayes pour les données sans stopwords.
'''
stopwords_fr = stopwords.words('french')
text_clf = Pipeline([('vect', CountVectorizer(stop_words = stopwords_fr)),
                         #Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])

text_clf.fit(train_text, train_tags)
predicted = text_clf.predict(test_text)

print("Precision: " + str(precision_score(test_tags, predicted, average='macro')))
print("Recall: " + str(recall_score(test_tags, predicted, average='macro')))
print("F measure: " + str(f1_score(test_tags, predicted, average='macro')))

Precision: 0.8771562113217379
Recall: 0.48858162431867636
F measure: 0.49771607701991866


In [7]:
df = pd.read_csv("../Ressources/scenarii.csv", encoding="ISO-8859-1", sep=";", header=0)
tags = df['tag']
data = CountVectorizer(stop_words = stopwords_fr).fit_transform(df['texte']).toarray()
#Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
data = TfidfTransformer().fit_transform(data).toarray()

In [8]:
clf = MultinomialNB()
scoring = {'accuracy': 'accuracy',
           'prec': 'average_precision', 
           'recall': 'recall',
           'f1_macro': 'f1_macro'}
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'f1_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'recall_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'precision_macro')
print(np.mean(scores))

0.6232842369298792
0.5765270986323617
0.8538589573126124


### DATASET AVEC VERBES OCCURENCES/TF-IDF


In [9]:
train_text = pd.read_csv("../Ressources/data_verbs_only/training_matrix_with_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=0)
train_tags = pd.read_csv("../Ressources/data_verbs_only/training_tags_with_verbs.csv", 
                         encoding="ISO-8859-1", sep=",", header = None)  
test_text = pd.read_csv("../Ressources/data_verbs_only/testing_text_with_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=None)
test_tags = pd.read_csv("../Ressources/data_verbs_only/testing_tags_with_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=None)

In [10]:
'''
Construction du modèle Naive Bayes pour les données avec les verbes uniquement.
'''
text_clf = Pipeline([
                         #Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
text_clf.fit(train_text, train_tags)
predicted = text_clf.predict(test_text)
print("Precision: " + str(precision_score(test_tags, predicted, average='macro')))
print("Recall: " + str(recall_score(test_tags, predicted, average='macro')))
print("F measure: " + str(f1_score(test_tags, predicted, average='macro')))

Precision: 0.6609297725024728
Recall: 0.31890965550653677
F measure: 0.3425557889030485


#### CROSS VALIDATION



In [11]:
data = pd.read_csv("../Ressources/data_verbs_only/all_data_with_verbs.csv", 
                   encoding="ISO-8859-1", sep=";", header=0)

tags = pd.read_csv("../Ressources/data_verbs_only/all_tags.csv", 
                   encoding="ISO-8859-1", sep=";", header=None)

In [12]:
#Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
data = TfidfTransformer().fit_transform(data).toarray()

In [13]:
'''
Construction du modèle Naive Bayes pour les données avec les verbes uniquemet en utilisant la cross validation.
'''
clf = MultinomialNB()
scoring = {'prec': 'average_precision', 
           'recall': 'recall',
           'f1_macro': 'f1_macro'}
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'f1_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'recall_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'precision_macro')
print(np.mean(scores))

0.3546348255076156
0.3241959013332787
0.6627837220329535


### DATASET AVEC VERBES D'ACTION OCCURENCES/TF-IDF

In [14]:
train_text = pd.read_csv("../Ressources/data_action_verbs_only/training_matrix_with_action_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=0)
train_tags = pd.read_csv("../Ressources/data_action_verbs_only/training_tags_with_action_verbs.csv", 
                         encoding="ISO-8859-1", sep=",", header = None)  
test_text = pd.read_csv("../Ressources/data_action_verbs_only/testing_text_with_action_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=None)
test_tags = pd.read_csv("../Ressources/data_action_verbs_only/testing_tags_with_action_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=None)

In [15]:
'''
Construction du modèle Naive Bayes pour les données avec les verbes d'action uniquement.
'''
text_clf = Pipeline([
                         #Commenter cette ligne pour avoir les occurences, et enlever le commentaire pour avoir les scores TF-IDF
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
text_clf.fit(train_text, train_tags)
predicted = text_clf.predict(test_text)
print("Precision: " + str(precision_score(test_tags, predicted, average='macro')))
print("Recall: " + str(recall_score(test_tags, predicted, average='macro')))
print("F measure: " + str(f1_score(test_tags, predicted, average='macro')))

Precision: 0.6834972911320216
Recall: 0.31998003785834367
F measure: 0.34045399668098647


#### CROSS VALIDATION

In [16]:
data = pd.read_csv("../Ressources/data_action_verbs_only/all_data_with_action_verbs.csv", 
                   encoding="ISO-8859-1", sep=",", header=0)

tags = pd.read_csv("../Ressources/data_action_verbs_only/all_tags.csv", 
                   encoding="ISO-8859-1", sep=",", header=0)

In [17]:
'''
Construction du modèle Naive Bayes pour les données avec les verbes d'action uniquemet en utilisant la cross validation.
'''
clf = MultinomialNB()
scores = cross_val_score(clf, data, tags, cv=5, scoring='f1_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'recall_macro')
print(np.mean(scores))
scores = cross_val_score(clf, data, tags, cv=5, scoring = 'precision_macro')
print(np.mean(scores))

0.4748317554476883
0.4200975735534166
0.6840339766891782
